In [1]:
#Import libraries
from scipy.sparse import diags
import scipy.sparse as sp
import scipy.sparse.linalg as sl
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.integrate

%run ../Utilities/Utilities.ipynb

## Poisson eq: Dirichlet left, Neumann right

In [2]:
def poissonSolve_DN(m, alpha, sigma, function):
    '''
    Function to solve with m steps, Dirichlet left and Neumann right
    '''
    #Constants
    M=m
    h=1/(M+1)

    #Make subdiagonal
    sub = np.ones(M)
    sub[-1] = 2*h

    #Make subsubdiagonal
    subsub = np.zeros(M-1)
    subsub[-1] = -h/2

    #Make superdiagonal
    sup = np.ones(M)

    #Make diagonal
    diag = -2*np.ones(M+1)
    diag[-1] = -3*h/2

    #Construct matrix
    A = diags([sup,diag,sub,subsub], offsets=[1,0,-1,-2], format = "csc")/h**2

    #Evaluate values
    xaxis = np.linspace(0, 1, M+2)
    
    fvals = function(xaxis[1:])
    fvals[0] -= alpha/h**2
    fvals[-1] = sigma
    
    y = np.zeros(M+2)
    y[1:] = sl.spsolve(A, fvals)
    y[0] = alpha
    
    #Solve
    return xaxis,y


## Poisson eq: Dirichlet left, Dirichlet right

In [3]:

def poissonSolve_DD(m,alpha,beta,function):
    '''
    Function that solves poisson equation with dirichlet conditions at both ends
    '''
    
    #Constants
    M=m
    h=1/(M+1)

    #Make subdiagonal
    sub = np.ones(M-1)
    
    #Make superdiagonal
    sup = np.ones(M-1)

    #Make diagonal
    diag = -2*np.ones(M)

    #Construct matrix
    A = diags([sup,diag,sub], offsets=[1,0,-1], format = "csc")/h**2

    #Evaluate values
    x_axis = np.linspace(0,1,M+2)
    fvals = function(x_axis[1:M+1])
    
    fvals[0] -= alpha/h**2
    fvals[-1] -= beta/h**2
    
    #Solve
    y = np.zeros(M+2)
    y[1:M+1] = sl.spsolve(A,fvals)
    y[0]=alpha; y[-1]=beta
    
    return x_axis, y



## Poisson eq: Neumann left, Neumann right

In [4]:
def poissonSolve_NN(m,sigma1,sigma2,function, simple = False):
    '''
    Function that attemps to solve the poisson equation with neumann conditions at both sides
    '''
    
    #Constants
    M=m
    h=1/(M+1)
    
    if not simple:
        try:
            #Make subdiagonal
            sub = np.ones(M+1)
            sub[-1] = 2*h

            #Make subsubdiagonal
            subsub = np.ones(M) 
            subsub[-1] = -h/2

            #Make superdiagonal
            sup = np.ones(M+1)
            sup[0] = 2*h

            #Make supersuperdiagonal
            supsup = np.ones(M)
            supsup[0] = -h/2

            #Make diagonal
            diag = -2*np.ones(M+2)
            diag[0] = -3*h/2
            diag[-1] = -3*h/2

            #Construct matrix
            A = diags([supsup, sup,diag,sub,subsub], offsets=[2, 1,0,-1,-2]).toarray()/h**2

            #Evaluate values
            xaxis = np.arange(0,1+1/(M+1),1/(M+1))[0:M+2]
            fvals = function(xaxis)

            fvals[0] = sigma1
            fvals[-1] = sigma2

            #Solve
            return xaxis,np.linalg.solve(A,fvals)
        
        except Exception as e:
            print(e)
            
    else:
        try: 
            #Make subdiagonal
            sub = np.ones(M+1)
            sub[-1] = h

            #Make superdiagonal
            sup = np.ones(M+1)
            sup[0] = h

            #Make diagonal
            diag = -2*np.ones(M+2)
            diag[0] = -h
            diag[-1] = -h

            #Construct matrix
            A = diags([sup,diag,sub], offsets=[1,0,-1]).toarray()/h**2

            #Evaluate values
            xaxis = np.arange(0,1+1/(M+1),1/(M+1))[0:M+2]
            fvals = function(xaxis)

            fvals[0] = sigma1 + h*fvals[0]/2
            fvals[-1] = sigma2 + h*fvals[-1]/2
        
        except Exception as e:
            print(e)

        #Solve
        return xaxis,np.linalg.solve(A,fvals)

## Poisson eq: Dirichlet-Dirichlet 1. order

In [5]:
def poissonSolve_DD1(M, alpha, beta, function, start=0, end=1):
    
    #Constants
    h = 1/(M+1)
    x_axis = np.linspace(0,1,M+2)
    
    #Make subdiagonal
    sub = np.ones(M-1)
    
    #Make superdiagonal
    sup = np.ones(M-1)
    
    #Make diagonal
    diag = -2*np.ones(M)
    
    #Construct matrix
    A = diags([sup,diag, sub], offsets=[1,0,-1], format="csc")/h**2
    
    #Evaluate values
    fvals = function(x_axis[0:M])
    
    fvals[0] -= alpha/h**2
    fvals[-1] -= beta/h**2
    
    #solve
    y=np.zeros(M+2)
    y[1:M+1] = sl.spsolve(A,fvals)
    y[0] = alpha; y[-1] = beta
    
    return x_axis,y

# Poisson eq: Dirichlet - Dirichlet AMR

In [6]:
import random

def make_abc_2nd_order(x): #x is x_axis
    
    M = len(x)-2
    
    h = [x[i+1]-x[i] for i in range(M+1)]

    a = np.zeros(M+2)   #empty until 3rd element
    b = np.zeros(M+2)   #2nd element is Beta
    c = np.zeros(M+2)   #2nd element is gamma
    
    b[1] = 1/h[0]**2
    c[1] = 1/h[0]**2

    for i in range(1,M+1):
        
        d_1 = h[i-2]+h[i-1]
        d_2 = h[i-1]
        d_3 = h[i]

        a[i] = 2*(d_3 - d_2) / (d_1*(d_1+d_3)*(d_1-d_2))
        b[i] = 2*(d_1 - d_3) / (d_2*(d_1-d_2)*(d_2+d_3))
        c[i] = 2*(d_1 + d_2) / (d_3*(d_2+d_3)*(d_1+d_3))

    
    return a,b,c
    
def make_abc_1st_order(x): #x is x_axis
    
    M = len(x)-2
    
    h = [x[i+1]-x[i] for i in range(M+1)]

    a = np.zeros(M+2)   #empty until 3rd element
    b = np.zeros(M+2)   #2nd element is Beta
    c = np.zeros(M+2)   #2nd element is gamma
    
    b[1] = 1/h[0]**2
    c[1] = 1/h[0]**2

    for i in range(1,M+1):

        
        d_1 = h[i-2]+h[i-1]
        d_2 = h[i-1]
        d_3 = h[i]


        b[i] = 1/(d_2*h[i])
        c[i] = 1/(d_3*h[i])

    
    return a,b,c

In [7]:
def AMR_DD(make_abc, startM, tol, dir1, dir2, fasit_fnc, error_method="max", maxiter=10000):
    
    #Information for error-plots
    statistics=[]
    
    #Starting with a uniform grid
    x_axis = np.linspace(0,1,startM)
    
    errors=[1000]
    M=startM
    
    
    while tol<sum(errors):
        #Making the coefficients for the matrix A
        a,b,c = make_abc(x_axis)

        #Construct matrix
        A = diags([c[1:-2], -(a+b+c)[1:-1] , b[2:-1] , a[3:-1]], offsets=[1,0,-1,-2], format = "csc")

        #Declaring 
        fvals = fxx(x_axis[1:-1])
        fvals[0]  -= b[1]*dir1
        fvals[1]  -= a[2]*dir1
        fvals[-1] -= c[-2]*dir2

        #Solve
        y = np.zeros(len(x_axis))
        y[1:-1] = sl.spsolve(A,fvals)
        y[0]=dir1; y[-1]=dir2
        
        #Making piecewise linear interpolation-function for the numerical solution y
        interpol_y = scipy.interpolate.CubicSpline(x_axis,y)
        
        #Initializing vectors for the errors and the new points
        errors = np.zeros(M-1)
        points_new = np.zeros(M-1)
        
        #Finding the errors in each point by using simpsons method on the difference between analytic solution and y_interpol
        for i in range(M-1):
            x_new = np.linspace(x_axis[i],x_axis[i+1],1000)
            errors[i] = scipy.integrate.simps((interpol_y(x_new)-fasit_fnc(x_new))**2, x_new)
    
        #Deciding which intervals to split
        limit_coeff = 0.7*max(abs(errors)) if (error_method == "max") else np.mean(errors)
        for i in range(M-1):
            if errors[i] > limit_coeff:
                points_new[i]= (x_axis[i]+x_axis[i+1])/2
        
        #For statistics
        x_new = np.linspace(0,1,10**4)
        l2_relative = np.sqrt(sum((fsol(x_axis)-y)**2) / sum(fsol(x_axis)**2))
        L2_relative = np.sqrt(scipy.integrate.simps((fsol(x_new)-interpol_y(x_new))**2)/scipy.integrate.simps(fsol(x_new)**2))
        statistics.append([M,l2_relative, L2_relative])
        
        #2nd order: For central differences to work, we must have h[0]=h[1].
        if  make_abc==make_abc_2nd_order and points_new[0]==0 and points_new[1]!=0:
            points_new[0] = (x_axis[0]+x_axis[1])/2
        
        #Security measure
        if M>maxiter: break
    
        #Combining the old x_axis with the new points
        x_axis = np.concatenate((x_axis, points_new[points_new!=0]))
        x_axis.sort(kind="mergesort")
        M = len(x_axis)
        
    
    return x_axis,y, np.array(statistics)


In [8]:
def errorPlot_AMR(statistics,order):
    plt.loglog(statistics[:,0], statistics[:,1],"-",label="$\ell_2$-norm",marker=".")
    plt.loglog(statistics[:,0], statistics[0,1]*statistics[0,0]**order/statistics[:,0]**order, "--",label="Expected order")
    plt.loglog(statistics[:,0], statistics[:,2], "-",label="$L_2$-norm",marker=".")
    plt.loglog(statistics[:,0], statistics[0,2]*statistics[0,0]**order/statistics[:,0]**order, "--",label="Expected order")
    
    plt.xlabel("Number of points along x-axis: $M$")
    plt.ylabel("Relative error $e^r_{(\cdot)}$")

    #plt.title("Error plot: 1st order FDM, AMR, average")
    plt.grid()
    plt.legend()
    plt.show()
    return 0

## Norms

In [9]:
def integralNorm(function,axis):
    return np.sqrt(scipy.integrate.simps(function**2,axis))

def l2norm(values):
    sum_sq = sum(values**2)
    return np.sqrt( sum_sq / len(values))